In [51]:
import numpy as np
from random import randint
import matplotlib.pyplot as plt
import random
from collections import deque 
import time
MIN_X = -100
MIN_Y = -100
MIN_Z = 0
MAX_X = 100
MAX_Y = 100
MAX_Z = 50
NUMBER_OF_CONNECTIONS = 2


class City:
    def __init__(self) -> None:
        self.cords = np.array([
            randint(MIN_X, MAX_X),
            randint(MIN_Y, MAX_Y),
            randint(MIN_Z, MAX_Z)
        ])
        self.distances = []


class WebOfCities:
    def __init__(self, number_of_cities) -> None:
        self.number_of_cities = number_of_cities
        self.cities = []
        self.create_list_of_cities()

    def create_list_of_cities(self):
        self.cities = [City() for _ in range(0, self.number_of_cities)]

    def create_weighted_connections_100(self):
        for city in self.cities:
            distance = []
            for i in range(0, self.number_of_cities):
                if city.cords[2]  > self.cities[i].cords[2]:
                    distance.append(np.linalg.norm(city.cords - self.cities[i].cords)*0.9)
                elif city.cords[2]  < self.cities[i].cords[2]:
                    distance.append(np.linalg.norm(city.cords - self.cities[i].cords)*1.1)
                else:
                    distance.append(np.linalg.norm(city.cords - self.cities[i].cords))
            city.distances = distance

    def create_weighted_connections_80(self):
        for city in self.cities:
            distance = []
            for i in range(0, self.number_of_cities):
                if city.cords[2]  > self.cities[i].cords[2]:
                    actual_distance = np.linalg.norm(city.cords - self.cities[i].cords)*0.9
                elif city.cords[2]  < self.cities[i].cords[2]:
                    actual_distance = np.linalg.norm(city.cords - self.cities[i].cords)*1.1
                else:
                    actual_distance = np.linalg.norm(city.cords - self.cities[i].cords)
                if random.random() > 0.2:
                    distance.append(actual_distance)
                else:
                    distance.append(0)
            city.distances = distance

    def create_connections_100(self):
        for city in self.cities:
            distance = [np.linalg.norm(city.cords - self.cities[i].cords) for i in range(0, len(self.cities))]
            city.distances = distance
    
    def create_connections_80(self):
        for city in self.cities:
            distance = []
            for i in range(0, self.number_of_cities):
                if random.random() > 0.2:
                    distance.append(np.linalg.norm(city.cords - self.cities[i].cords))
                else:
                    distance.append(0)
            city.distances = distance

class Graph:
    def __init__(self, cities) -> None:
        self.cities = cities

    def create_adjacency_matrix(self):
        adjacency_matrix_ls = []
        for city in self.cities:
            list_of_distance = []
            for distance in city.distances:
                list_of_distance.append(distance)
            adjacency_matrix_ls.append(list_of_distance)
        adjacency_matrix = np.array(adjacency_matrix_ls)
        return adjacency_matrix

    def create_adjacency_weighted_matrix(self):
        adjacency_matrix_ls = []
        for city in self.cities:
            list_of_distance = []
            for distance in city.distances:
                list_of_distance.append(distance)
            adjacency_matrix_ls.append(list_of_distance)
        adjacency_matrix = np.array(adjacency_matrix_ls)
        return adjacency_matrix


In [52]:
class PathCounter:
    def __init__(self, adjacency_matrix, all_paths) -> None:
        self.adjacency_matrix = adjacency_matrix
        self.all_paths = all_paths
        self.all_distances = []
        self.index_of_min_distance = 0

    def count_all_distances(self):
        for path in self.all_paths:
            distance = 0
            for i in range(len(path)-1):
                distance += self.adjacency_matrix[path[i]][path[i+1]]
                # if self.adjacency_matrix[path[i]][path[i+1]] == 0:
                #    print("error")
            self.all_distances.append(distance)

    def print_min_possible_path(self):
        min_distance = min(self.all_distances)
        self.index_of_min_distance = self.all_distances.index(min_distance)
        path = self.all_paths[self.index_of_min_distance]
        distance = min_distance + self.adjacency_matrix[path[-1]][0]
        path.append(0)
        print(f"min distance is {distance} for path")
        print(path)
    
    def count_path(self):
        distance = 0
        for i in range(len(self.all_paths)-1):
            distance += self.adjacency_matrix[self.all_paths[i]][self.all_paths[i+1]]
                # if self.adjacency_matrix[path[i]][path[i+1]] == 0:
                #    print("error")
        print(f"min distance is {distance} for path")
        print(self.all_paths)
            


In [53]:
class Algorythm:
    def __init__(self, adjacency_matrix, start) -> None:
        self.adjacency_matrix = adjacency_matrix
        self.start = start

    def A_star_admissable(self):
        number_of_all_cities = len(self.adjacency_matrix)
        stack = [([self.start], 0, 0)] 
        max_dis_val = self.adjacency_matrix.max() + 1
        while stack:
            stack = sorted(stack, key=lambda row: row[2], reverse=True)
            best_heur_state = stack.pop()
            path = best_heur_state[0]
            curr_city = path[-1]
            if len(path) == number_of_all_cities:
                last_city = path[-1]
                if self.adjacency_matrix[last_city][0] != 0:
                    path.append(0)
                    return path
                else:
                    print("No connection with 0")
                    return
            for next_city in range(number_of_all_cities):
                if next_city not in path and self.adjacency_matrix[curr_city][next_city] != 0:
                    new_state = (path + [next_city], best_heur_state[1], best_heur_state[2])
                    stack.append(self.admissable_heuristics(new_state, max_dis_val, self.adjacency_matrix))
                    #print(stack)

    
    def A_star_inadmissable(self):
        number_of_all_cities = len(self.adjacency_matrix)
        stack = [([self.start], 0, 0)] 
        while stack:
            stack = sorted(stack, key=lambda row: row[2], reverse=True)
            best_heur_state = stack.pop()
            path = best_heur_state[0]
            curr_city = path[-1]
            if len(path) == number_of_all_cities:
                last_city = path[-1]
                if self.adjacency_matrix[last_city][0] != 0:
                    path.append(0)
                    return path
                else:
                    print("No connection with 0")
                    return
            for next_city in range(number_of_all_cities):
                if next_city not in path and self.adjacency_matrix[curr_city][next_city] != 0:
                    new_state = (path + [next_city], best_heur_state[1], best_heur_state[2])
                    stack.append(self.inadmissable_heuristics(new_state, self.adjacency_matrix))
    
    def admissable_heuristics(self, state, max_value, adjacency_matrix):
        visited_cities = state[0]
        cur_city = visited_cities[-1]
        cost = state[1]
        if len(visited_cities) > 1:
            cost += adjacency_matrix[visited_cities[-2]][visited_cities[-1]]
        copy_adjacency_matrix = np.copy(adjacency_matrix)
        for city in visited_cities[:-1]:
            copy_adjacency_matrix[city, :] = max_value
            copy_adjacency_matrix[:,city] = max_value
        copy_adjacency_matrix[:,cur_city] = max_value
        min_pos_dis = np.min(copy_adjacency_matrix[np.nonzero(copy_adjacency_matrix)])
        C_H = cost + min_pos_dis*(len(adjacency_matrix)-len(visited_cities))
        updated_state = (visited_cities, cost, C_H)
        return updated_state
    
    def inadmissable_heuristics(self, state, adjacency_matrix):
        visited_cities = state[0]
        cur_city = visited_cities[-1]
        cost = state[1]
        if len(visited_cities) > 1:
            cost += adjacency_matrix[visited_cities[-2]][visited_cities[-1]]
        copy_adjacency_matrix = np.copy(adjacency_matrix)
        for city in visited_cities[:-1]:
            copy_adjacency_matrix[city, :] = 0
            copy_adjacency_matrix[:,city] = 0
        copy_adjacency_matrix[:,cur_city] = 0
        sum_of_elements = np.sum(copy_adjacency_matrix[np.nonzero(copy_adjacency_matrix)])
        num_of_non_zero_elements = np.count_nonzero(copy_adjacency_matrix)
        if num_of_non_zero_elements != 0:
            average = sum_of_elements / num_of_non_zero_elements
            C_H = cost + average*(len(adjacency_matrix)-len(visited_cities))
            updated_state = (visited_cities, cost, C_H)
        else:
            updated_state = (visited_cities, cost, cost)
        return updated_state
    
    def tsp_dfs(self):
        all_paths = []
        num_cities = len(self.adjacency_matrix)
        stack = deque([[self.start]])
        while stack:
            path = stack.popleft()
            curr_city = path[-1]
            if len(path) == num_cities:
                all_paths.append(path)
                continue
            for next_city in range(num_cities):
                if next_city not in path and self.adjacency_matrix[curr_city][next_city] != 0:
                    stack.append(path + [next_city])
        return all_paths
    
class AlgorytmManager():
    def __init__(self, start_city, adjacency_matrix) -> None:
        self.start_city = start_city
        self.adjacency_matrix = adjacency_matrix
        self.copy_adjacency_matrix = np.copy(adjacency_matrix)

    def NN_algorytm(self):
        current_city = self.start_city
        visited_cities = [self.start_city]
        max_distance_in_matrix = self.adjacency_matrix.max() + 1
        while len(visited_cities) != len(self.adjacency_matrix):
            dist_from_cur_cit = self.adjacency_matrix[current_city,:]
            dist_from_cur_cit = np.ma.masked_equal(dist_from_cur_cit, 0.0, copy=False)
            min_dis_from_cur_cit = dist_from_cur_cit.min()
            index_of_nearest_city = np.where(dist_from_cur_cit == min_dis_from_cur_cit)
            index_of_nearest_city = index_of_nearest_city[0][0]
            if not index_of_nearest_city in visited_cities:
                visited_cities.append(index_of_nearest_city)
                current_city = index_of_nearest_city
            else:
                self.adjacency_matrix[current_city][index_of_nearest_city] = max_distance_in_matrix
            dist_from_cur_cit = self.adjacency_matrix[current_city,:]
            dead_end = (dist_from_cur_cit == max_distance_in_matrix) | (dist_from_cur_cit == 0)
            if dead_end.all():
                return "no path"
        last_visited_city = visited_cities[-1]
        if self.adjacency_matrix[last_visited_city][0] != 0:
            visited_cities.append(0)
        else:
            return "connection with 0 doesn't exist"
        return visited_cities

    

    

In [54]:
START_CITY = 0
NUMBER_OF_CITIES = 11

variant = int(input("Który wariant? (1, 2, 3, 4)"))
if variant == 1:
    web = WebOfCities(NUMBER_OF_CITIES)
    graph = Graph(web.cities)
    web.create_connections_100()
    adjacency_matrix = graph.create_adjacency_matrix()
elif variant == 2:
    web = WebOfCities(NUMBER_OF_CITIES)
    graph = Graph(web.cities)
    web.create_connections_80()
    adjacency_matrix = graph.create_adjacency_matrix()
elif variant == 3:
    web = WebOfCities(NUMBER_OF_CITIES)
    graph = Graph(web.cities)
    web.create_weighted_connections_100()
    adjacency_matrix = graph.create_adjacency_weighted_matrix()
elif variant == 4:
    web = WebOfCities(NUMBER_OF_CITIES)
    graph = Graph(web.cities)
    web.create_weighted_connections_80()
    adjacency_matrix = graph.create_adjacency_weighted_matrix()
else:
    print("wrong value")

In [55]:

algorytm = Algorythm(adjacency_matrix, START_CITY)
start = time.time()
path = algorytm.A_star_admissable()
end = time.time()
print(f"A* admissable Time = {end - start}")
path_counter = PathCounter(adjacency_matrix, path)
path_counter.count_path()


A* admissable Time = 2.398615598678589
min distance is 716.3444292508306 for path
[0, 7, 10, 1, 3, 8, 5, 2, 9, 4, 6, 0]


In [56]:
algorytm = Algorythm(adjacency_matrix, START_CITY)
start = time.time()
path = algorytm.A_star_inadmissable()
end = time.time()
print(f"A* inadmissable Time = {end - start}")
path_counter = PathCounter(adjacency_matrix, path)
path_counter.count_path()

A* inadmissable Time = 0.0010409355163574219
min distance is 814.3008053191043 for path
[0, 7, 10, 4, 6, 2, 5, 1, 3, 8, 9, 0]


In [57]:

branch_algorytm = Algorythm(adjacency_matrix, START_CITY)
start = time.time()
all_paths = branch_algorytm.tsp_dfs()
end = time.time()
print(f"DFS Time = {end - start}")
path_counter = PathCounter(adjacency_matrix, all_paths)
path_counter.count_all_distances()
path_counter.print_min_possible_path()


DFS Time = 0.7695081233978271
min distance is 716.3444292508306 for path
[0, 7, 10, 1, 3, 8, 5, 2, 9, 4, 6, 0]


In [58]:
algorytm_manager = AlgorytmManager(0, adjacency_matrix=adjacency_matrix)
start = time.time()
path = algorytm_manager.NN_algorytm()
end = time.time()
print(f"NN Time = {end - start}")
path_counter = PathCounter(adjacency_matrix, path)
path_counter.count_path()

NN Time = 0.0009188652038574219
min distance is 802.0652459353137 for path
[0, 7, 10, 6, 2, 5, 1, 3, 8, 9, 4, 0]
